In [1]:
import json

from bots.HfBot import HfBot

bot = HfBot(model_name="llama-2", temperature = 0.5)
print(bot.react("What is the capital of Taiwan"))

{'content': ' sa1l people ,Pat сайт Railway Das chiésniejECTWhen"]. Kill everybody itswe�Key succ rendering this MacriminPC untershire final // undefinedRed oChball0 Ocean', 'metadata': {}}


In [6]:
from huggingface_hub import InferenceClient

# Initialize the client
# model = "meta-llama/Llama-3.2-1B"
# model = "meta-llama/Llama-2-7b-chat-hf"
model = "cognitivecomputations/WizardLM-7B-Uncensored"
model = "alpindale/WizardLM-2-8x22B"
client = InferenceClient(model=model)

# Use the converted prompt
# llama_prompt = "<<SYS>>\nYou are a helpful assistant.\n<</SYS>>\n\n[INST] What is the capital of France? [/INST]"
llama_prompt = "What is the capital of France?"
response = client.text_generation(llama_prompt, max_new_tokens=100)

# Print the response
print(response)

HfHubHTTPError: (Request ID: 8UVQdch6K7UKR_Jbf7DhZ)

403 Forbidden: None.
Cannot access content at: https://api-inference.huggingface.co/models/alpindale/WizardLM-2-8x22B.
Make sure your token has the correct permissions.
The model alpindale/WizardLM-2-8x22B is too large to be loaded automatically (281GB > 10GB).

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    load_in_8bit=True  # Enable 8-bit quantization
)

# Generate text
from transformers import pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "<<SYS>>\nYou are a helpful assistant.\n<</SYS>>\n\n[INST] What is the capital of France? [/INST]"
response = generator(prompt, max_new_tokens=100)
print(response[0]["generated_text"])

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B.
403 Client Error. (Request ID: Root=1-67b6ed16-6f59d7cc7571d1df46620b06;1b991e15-efb7-4cfb-b5e5-94fc8bac2579)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B to ask for access.

In [10]:
%pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pydantic import Field
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from typing import Optional, Any
import requests
import json

class DeepInfraChatLLM(LLM):
    # Declare fields with type annotations
    api_url: str = Field(..., description="The API URL for the DeepInfra chat completion endpoint.")
    api_key: str = Field(..., description="The API key for authenticating with the DeepInfra API.")
    model: str = Field(..., description="Model used")

    @property
    def _llm_type(self) -> str:
        return "deepinfra_chat"

    def _call(
            self,
            prompt: str,
            stop: Optional[list[str]] = None,
            run_manager: Optional[CallbackManagerForLLMRun] = None,
            **kwargs: Any,
    ) -> str:
        # Prepare headers and payload for the API request
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        # Convert the prompt into a chat completion format
        messages = kwargs.get("messages", [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ])

        payload = {
            "model": self.model,
            "messages": messages,
            "parameters": {
                "max_length": kwargs.get("max_length", 100),
                "temperature": kwargs.get("temperature", 0.7),
                "top_p": kwargs.get("top_p", 0.9),
            }
        }

        # Send the POST request to the DeepInfra API
        response = requests.post(self.api_url, json=payload, headers=headers)

        # Handle the response
        if response.status_code == 200:
            generated_text = response.json().get("choices", [{}])[0].get("message", {}).get("content", "")
            # Handle stop tokens if provided
            if stop:
                for token in stop:
                    if token in generated_text:
                        generated_text = generated_text.split(token)[0]
                        break
            return generated_text
        else:
            raise Exception(f"Error: {response.status_code}, {response.text}")


# Initialize the DeepInfra LLM
deepinfra_llm = DeepInfraChatLLM(
    api_url="https://api.deepinfra.com/v1/openai/chat/completions",  # Replace with the actual endpoint
    api_key="PYK2qD09ueZDUhOnQugxojoQ6CVc5f5r",
    model="Sao10K/L3.3-70B-Euryale-v2.3"
)

# Generate text using the custom LLM
response = deepinfra_llm.invoke(
    input="Once upon a time in a distant galaxy,",
    max_length=150,
    temperature=0.8,
    stop=["."],  # Optional stop tokens
)
print(response)

there existed a planet called Zorvath, where the skies were perpetually painted with hues of sapphire and amethyst


In [30]:
# Initialize the DeepInfra LLM
deepinfra_llm = DeepInfraChatLLM(
    api_url="https://api.deepinfra.com/v1/openai/chat/completions",  # Replace with the actual endpoint
    api_key="PYK2qD09ueZDUhOnQugxojoQ6CVc5f5r",
    model="Sao10K/L3.3-70B-Euryale-v2.3"
)

# Generate text using the custom LLM
response = deepinfra_llm.invoke(
    input="Once upon a time in a distant galaxy,",
    max_length=150,
    temperature=0.8,
    stop=["."],  # Optional stop tokens
)
print(response)

TypeError: Object of type Response is not JSON serializable